<br>
<h1 style="color:green;text-align:center;">
    End of Course Assessment (ECA)
</h1>
<hr>
<br>
<br>

<br>
<h2 style="color:blue;text-align:center;">
    Question 1
</h2>
<br>

<h5>>>> 1(a) + 1(b)</h5>

<b>Requirements:</b> <br>
Using IRAS website: https://www.iras.gov.sg/irashome/Other-Taxes/Stamp-Duty-forProperty/Working-out-your-StampDuty/Buying-or-Acquiring-Property/Whatis-the-Duty-that-I-Need-to-Pay-as-a-Buyer-or-Transferee-of-ResidentialProperty/Buyer-s-Stamp-Duty--BSD-/ as of July 2019
<br><br>

In [24]:
from abc import ABC , abstractmethod

class Buyer(ABC):
    """
    Abstract Class for buyers
    """
    #Constructor for the class
    def __init__(self,buyerId,name,contact):
        self._buyerId = buyerId
        self._name = name
        self._contact = contact
        self._numberOfProperties = 0

    @property
    def buyerId(self):  #returns buyers id
        return self._buyerId
    @property
    def contact(self):  #return buyers contact number
        return self._contact
    @contact.setter    #modify buyers contact number
    def contact(self,newValue):
        self._contact = newValue
    @property          #returns number of Properties buyer owns
    def numberOfProperties(self):
        return self._numberOfProperties
    @numberOfProperties.setter #modify number of Properties buyer owns
    def numberOfProperties(self,newValue):
        self._numberOfProperties = newValue
    @abstractmethod    #abstract method for Additional buyer stamp duty rate
    def getABSDRate(self):
        pass
    @classmethod
    @abstractmethod  #abstract class method for getting buyer residency status
    def getCitizenship(cls):
        pass

    def __str__(self):
        return f"Buyer Citizen: {type(self).getCitizenship():<4} id: {self._buyerId:<8} Name: {self._name:<8} "\
    f"Contact: {self._contact:<8} Number Of Properties: {self._numberOfProperties}"

class SingaporeCitizen(Buyer):
    """
    Buyers who are citizens of Singapore. Inhwrits from abstract Buyer class.
    """

    def getABSDRate(self): #return Additional buyer stamp duty rate
        if self.numberOfProperties<=1:   #if buyer owns 0 or 1 properties ABSD rate is 0
            return 0
        elif self.numberOfProperties==2: #if buyer owns 2 properties ABSD rate is 0.12
            return 0.12
        else:
            return 0.15

    @classmethod
    def getCitizenship(cls):    #method for getting buyer residency status which is SC
        return 'SC'

class TransactedProperty:

    #Constructor for the Class
    def __init__(self,address,propertyValue):

        self._address = address
        self._propertyValue = propertyValue

    @property
    def address(self):    #returns property address
        return self._address

    @property
    def propertyValue(self):  #returns Value of property
        return self._propertyValue
    @propertyValue.setter     #modifies Value of property
    def propertyValue(self,newValue):
        self._propertyValue = newValue

    def __str__(self):
        return f"Property Address: {self._address:<10} Value: ${self._propertyValue}"

class Transaction:

    _nextTransactionId = 1   #running  transaction id number

    #Constructor for the Class
    def __init__(self,transactedProperty,buyer):

        self._buyer = buyer
        self._transactedProperty = transactedProperty
        self._transactionId = Transaction._nextTransactionId
        Transaction._nextTransactionId+=1   #increment the running transaction id number by 1
        self._buyer.numberOfProperties +=1  #increase the number of properties owned by the buyer by 1
        self._ABSDRate = self._buyer.getABSDRate()  #buyers ABSD rate


    @property
    def buyer(self):  #returns buyer object
        return self._buyer
    @property
    def transactedProperty(self):  #returns TransactedProperty object
        return self._transactedProperty
    @property
    def transactionId(self):      #returns transaction id
        return self._transactionId
    @property
    def buyerId(self):          #returns buyers id number
        return self._buyer.buyerId

    def ABSDPayable(self):    #returns computed ABSD amount that needs to be paid
        return self._ABSDRate * self._transactedProperty.propertyValue

    def BSDPayable(self):    #returns Buyer stamp duty
        propertyValue = self._transactedProperty.propertyValue
        if propertyValue<=180000:
            return 0.01*propertyValue
        elif 180000<propertyValue<=360000:  #if property value is within $180,000-$360,000 value
            return ((0.01*180000) + (0.02*(propertyValue-180000)))
        elif 360000<propertyValue<=1000000:  #if property value is within $360,000-$1,000,000 value
            return ((0.01*180000) + (0.02*180000) + (0.03*(propertyValue-360000)))
        else:   #if property value is more than $1,000,000
            return ((0.01*180000) + (0.02*180000) + (0.03*640000) + (0.04*(propertyValue-1000000)))

    def __str__(self):
        return f"Transaction Id: {self._transactionId:<3} ABSD: ${self.ABSDPayable():0.2f} BSD: ${self.BSDPayable():0.2f}\n"\
    f"\t{self._transactedProperty}\n"\
    f"\t{self._buyer}"

class Registry:

    def __init__(self):

        self._buyers = {}    #collections of registered buyers
        self._transactions = []  #collections of registered transactions

    def locateBuyer(self,buyerId): #If buyer is registered it returns the buyer else return None
        try:
            return self._buyers[buyerId]
        except KeyError:
            return None

    def registerBuyer(self,buyer):

        if not self.locateBuyer(buyer.buyerId): #if buyer is not registered it register the buyers and retuens True
            self._buyers[buyer.buyerId] = buyer  # else returns False
            return True
        return False

    def removeBuyer(self,buyerId):

        if self.locateBuyer(buyerId):
            if self.locateBuyer(buyerId).numberOfProperties:
                return False
            else:
                self._buyers.pop(buyerId) #if buyer is registered and has 0 properties then remove the buyer
                return True               #and return True else return False
        return False

    def locateTransaction(self,transactionId): #If transaction is registered return the transaction else return None
        for properties in self._transactions:
            if properties.transactionId == transactionId:
                return properties
        return None

    def addTransaction(self,transaction):
        #If transaction is not registered then register the transaction & return True
        #else return False
        if not self.locateTransaction(transaction.transactionId):
            self._transactions.append(transaction)
            return True
        return False

    def transactionsStr(self):
        #if no transaction has been registered so far then we execute the following
        if not len(self._transactions):
            return 'Transaction List:\nNo transaction in registry.'
        else:
            #if transactions have been registered so far then we execute the following
            s='Transaction List:\n'
            for transaction in self._transactions:
                s+=f"{transaction}\n"
            return s

    def buyersStr(self):
        #if no buyer has been registered so far then we execute the following
        if not len(self._buyers):
            return 'Buyer List:\nNo buyer in registry.'
        else:
            #if buyers have been registered so far then we execute the following
            s='Buyer List:\n'
            buyers = sorted(self._buyers.values(),key=lambda x : x.numberOfProperties)
            for b in buyers:
                s+=f"{b}\n"
            return s

    def __str__(self):
        return f"{self.buyersStr()}\n\n{self.transactionsStr()}"



#----Example 1-------
def main():
    
    register_y = Registry()
    print(register_y)
main()    

#-----Example 2--------
def main():
    
    register_y = Registry()
    register_y.registerBuyer(SingaporeCitizen('T0001234X','Ann Chua',9213123))
    register_y.registerBuyer(SingaporeCitizen('T1234567F','Tom Teo',98712123))
    print(register_y)    
    
main()

#------Example 3----------------
def main():

    register_y = Registry()
    #Creating Buyers
    ann = SingaporeCitizen('T0001234X','Ann Chua',9213123)
    ismail = SingaporeCitizen('T0008989U','Ismail B',92888866)
    tom = SingaporeCitizen('T1234567F','Tom Teo',98712123)
    #Register Buyers
    register_y.registerBuyer(ann)
    register_y.registerBuyer(ismail)
    register_y.registerBuyer(tom)
    #Create Properties
    p1 = TransactedProperty('12 Tampines Road',1500000)
    p2 = TransactedProperty('14 Tampines Road',1500000)
    p3 = TransactedProperty('16 Tampines Road',1800000)
    #Create Transactions
    t1_tom = Transaction(p1,tom)
    t2_tom = Transaction(p2,tom)
    t1_ismail = Transaction(p3,ismail)
    #Register Transactions
    register_y.addTransaction(t1_tom)
    register_y.addTransaction(t2_tom)
    register_y.addTransaction(t1_ismail)
    #Print
    print(register_y)

main()

Buyer List:
No buyer in registry.

Transaction List:
No transaction in registry.
Buyer List:
Buyer Citizen: SC   id: T0001234X Name: Ann Chua Contact: 9213123  Number Of Properties: 0
Buyer Citizen: SC   id: T1234567F Name: Tom Teo  Contact: 98712123 Number Of Properties: 0


Transaction List:
No transaction in registry.
Buyer List:
Buyer Citizen: SC   id: T0001234X Name: Ann Chua Contact: 9213123  Number Of Properties: 0
Buyer Citizen: SC   id: T0008989U Name: Ismail B Contact: 92888866 Number Of Properties: 1
Buyer Citizen: SC   id: T1234567F Name: Tom Teo  Contact: 98712123 Number Of Properties: 2


Transaction List:
Transaction Id: 1   ABSD: $0.00 BSD: $44600.00
	Property Address: 12 Tampines Road Value: $1500000
	Buyer Citizen: SC   id: T1234567F Name: Tom Teo  Contact: 98712123 Number Of Properties: 2
Transaction Id: 2   ABSD: $180000.00 BSD: $44600.00
	Property Address: 14 Tampines Road Value: $1500000
	Buyer Citizen: SC   id: T1234567F Name: Tom Teo  Contact: 98712123 Number Of

<br>
<h5>>>> 1(c)</h5>

In [6]:
class PrivateTransaction(Transaction):
    
    #If buyer doesn't own any HDB then variable represnting it hdbDisposed will be None
    #and monthsDisposed will be None too
    #But if buyer has disposed HFB unit then they need to enter hdbDisposed
    #and monthsDisposed
    
    def __init__(self,transactedPrivateProperty,buyer,hdbDisposed=None,monthsDisposed=None):
        self._hdbDisposed = hdbDisposed  #HDB that has been disposed
        self._monthsDisposed = monthsDisposed   #Number of months it has been since it was disposed
        super().__init__(transactedPrivateProperty,buyer)
                
    def getRebate(self):
        #If no HDB unit has been bought then there will be rebate amount returned
        #which is 50% of transactioning property ABSD payable
        if not self._hdbDisposed:
            return 0.50 * self.ABSDPayable()
        else:
            #If HDB unit has been disposed but it has not been 18 month since its disposal
            # then there will be no rebate
            if self._monthsDisposed<18:
                return 0
            else:
                #If HDB unit has been disposed but it has been atleast 18 month since its disposal
                # then there will be rebate amount returned which is 50% of transactioning property ABSD payable
                return 0.50 * self.ABSDPayable()
    
    def __str__(self):
        
        total_absd_payables = self.ABSDPayable() - self.getRebate() #Actual ASBD payables after deducting rebate amount
        
        if self._hdbDisposed:  #If HDB was disposed it will show up in the print else it will not appear
            status = f"\n\tHDB Unit disposed address: {self._hdbDisposed.address}"
        else:
            status=''
            
        return f"Transaction Id: {self._transactionId:<3} ABSD: ${total_absd_payables:0.2f} BSD: ${self.BSDPayable():0.2f}\n"\
    f"\t{self._transactedProperty}\n"\
    f"\t{self._buyer}{status}"
  

<b><i style="text-decoration:underline;">Explaining polymorphism using this class as an example.</i></b><br><br>
Polymorphism means the same function defined in objects of different types will behave as intended. <br>
Using PrivateTransaction as an example, both PrivateTransaction class and Transaction class have __str__() method but they <br>behave as defined in their respective classes. In Transaction class there is no rebate amount where as in <br>PrivateTransaction class there is rebate amount present which will be deducted from the Additional buyer stamp duty <br>paybles in the __str__() method before being printed out.<br>
Hence, if there is a list of transactions containing all objects of both  PrivateTransaction class and Transaction class, when iterating over <br> the list to print the objects of the list, all objects will be making use of __str__() method but its implementation will be specific to its type class.
<br><br>




<h5>>>> 1(d)</h5>

The three <b>SOLID</b> principles I will be using are Single Responsibility Principle, Open-Closed Principle and Liskov Substitution Principle

<ul>
    <li><u>Single Responsibility Principle (SRP)</u><br>
        This Principle states that classes should have a single responsibility and thus only a single reason to change.
        This means a class should only serve one purpose. For example, in my program TransactedProperty class abides by SRP
        as it has only one task of creating property. Another example is of SingaporeCitizen class which also abides by SRP
        as it only creates buyers who are Singaporean. Both of these classes contain unique information specific to each.
        <br><br>
    </li>
    <li><u>Open-Closed Principle (OCP)</u><br>
        This principle states classes and other entities should be closed for modifications but open for extension. This 
        means you can extend the system without having to modify the existing code. For example, in my program Buyer class 
        abides by OCP as it is able to be extended to SingaporeCitizen class, SingaporePr class and Foreigner class without 
        making any modification to any aspect of original Buyer class code to accommodate for the sub-class.
        <br><br>
    </li>
    <li><u>Liskov Substitution Principle (LSP)</u><br>
        This principle states that the functions that use pointers to base classes must be able to use objects of derived 
        classes without knowing it. This also suggests that objects of a superclass shall be replaceable with objects of 
        its subclasses without breaking the application. For example, in my program SingaporeCitizen class abides by LSP as 
        it depends and makes use of all the methods inherited from its parent class Buyer to function wholly as a class.
        <br><br>
    </li>
</ul>

<br>
<h2 style="color:blue;text-align:center;">
    Question 2
</h2>
<br>

<h5>>>> 2(a) + 2(b) + 2(c)</h5>

<b>Requirements:</b> <br>
<p>
    deals with keeping track of 3 types of expenditures: food, transport and education.
</p>

In [26]:
from datetime import datetime , timedelta

class ExpenditureException(Exception):

    """
    Exception Class for Expenditure.
    Exception from this class is raised whenever there is an error when recording
    an expenditure or when retrieving expenditure record(s) from the expenditure list
    """

    def __init__(self,message,errorType):
        self._errorType = errorType
        super().__init__(message)

    @property
    def errorType(self):
        return self._errorType

class Expenditure:

    """
    Class records a single expenditure
    """

    def __init__(self,expenditureDate,amount,expenditureType):

        if amount<0:  #If amount is less than zero raise Exception
            raise ExpenditureException(f'Amount ${amount:0.2f} cannot be negative','Amount')
        elif amount==0:  #If amount is equals to zero raise Exception
            raise ExpenditureException(f'Amount ${amount:0.2f} cannot be zero','Amount')
        if expenditureDate > datetime.today():  #If the supplied date is later than today raises Exception.
            str_date = expenditureDate.strftime('%a, %d %b %Y')
            raise ExpenditureException(f"{str_date} cannot be later than today",'Date')
        self._amount = amount
        self._expenditureDate = expenditureDate
        self._expenditureType = expenditureType

    @property
    def amount(self):  #returns amount of expenditure
        return self._amount
    @property
    def expenditureDate(self):   #returns Expenditure Date
        return self._expenditureDate
    @property
    def expenditureType(self):   #returns Type of Expenditure
        return self._expenditureType

    def __str__(self):
        str_date = self._expenditureDate.strftime('%a, %d %b %Y')
        return f'${self._amount:0.2f} {str_date} {self._expenditureType}'

class ExpenditureList:


    _types = ['Food','Transport','Education']

    def __init__(self):
        self._expenditureList = []  #single collection of expenditures


    @classmethod
    def expenditureType(cls):  #returns list of Types of Expenditures which is class attribute
        return cls._types

    def getExpenditures(self,expenditureType,days=0):

        if days<0:  #if days are negative raise an exception
            raise ExpenditureException(f'Days {days} cannot be negative','Days')
        exp_lis = []
        if days==0:  #if days are 0 then all expenditures of the specified type are returned.
            #if expenditures in collection of expenditures are of type as in parameter then it gets
            #appended to exp_lis
            for expenditures in self._expenditureList:
                if expenditures.expenditureType == expenditureType:
                    exp_lis.append(expenditures)
        else:
            #if days are not 0 then Expenditures in n days from today, of the specified type are returned
            date_range = [(datetime.today().date()-timedelta(days=x)) for x in range(days)]
            #if expenditures in collection of expenditures are of same type as in parameter and are
            # within the days then it gets appended to exp_lis
            for expenditures in self._expenditureList:
                if expenditures.expenditureType == expenditureType and expenditures.expenditureDate.date() in date_range:
                    exp_lis.append(expenditures)
        return exp_lis

    def getExpendituresAmount(self,expenditureType,days=0):

        if days<0:             #if days are negative raise an exception
            raise ExpenditureException(f'Days {days} cannot be negative','Days')

        expenditure_list = self.getExpenditures(expenditureType,days) #Gets Expenditure List

        amount_list = [x.amount for x in expenditure_list]

        if expenditure_list==[]: #if its empty list then return 0 else compute the sum
            return 0
        else:
            return sum(amount_list)

    def addExpenditure(self,expenditureDate,amount,expenditureType):

        #if the supplied expenditure type is not one of the types stored in the class attribute _types
        #then raise Exception else begin the function of adding the successfully creating
        #Expenditure object

        if not expenditureType in type(self)._types:
            str_types = ', '.join(type(self)._types)
            raise ExpenditureException(f'Expenditure type {expenditureType} is not one of the valid type choices: {str_types}','Expenditure Type')
        self._expenditureList.append(Expenditure(expenditureDate,amount,expenditureType))
        return Expenditure(expenditureDate,amount,expenditureType)

    def __str__(self):

        s=""

        for expenditures in self._expenditureList:
            if expenditures.expenditureType=="Food":
                s+=f"{expenditures}\n"
        food = self.getExpendituresAmount('Food')
        s+=f"Total for Food: ${food:0.2f}\n"

        for expenditures in self._expenditureList:
            if expenditures.expenditureType=="Transport":
                s+=f"{expenditures}\n"
        trans = self.getExpendituresAmount('Transport')
        s+=f"Total for Transport: ${trans:0.2f}\n"
        for expenditures in self._expenditureList:
            if expenditures.expenditureType=="Education":
                s+=f"{expenditures}\n"
        edu = self.getExpendituresAmount('Education')
        s+=f"Total for Education: ${edu:0.2f}\n"
        total = 0
        for x in type(self)._types:
            total+=self.getExpendituresAmount(x)
        s+=f"Overall Total = ${total:0.2f}\n"
        return s

def getMenu():

    #Function returns the option

    while True:
        try:
            print("""Menu
1. Add Expenditure
2. List Expenditure by Type
3. List All Expenditures
0. Exit""")
            option = int(input('Enter choice:'))
            #If option is less than 0 or more than 3 continue else return option
            if option<0 or option>3:
                print('Please enter a number from 0 to 3 as a menu choice')
                continue
            return option
        except ValueError:   #If option is non-numerical entry then continue
            print('Please enter a number from 0 to 3 as a menu choice')

def Option1(Expenditure__list):

    try:
        while True:
            try:
                choice = int(input('Enter Expenditure type (1-> Food, 2-> Transport, 3-> Education):'))
                #If choice is not 1/2/3 then conintue else break out of this loop
                if choice!=1 and choice!=2 and choice!=3:
                    print('Type must be from 1 to 3')
                    continue
                if choice==1:
                    ex_type = 'Food'
                elif choice==2:
                    ex_type = 'Transport'
                else:
                    ex_type = 'Education'
                break
            except ValueError:   #If choice is non-numerical value then conintue
                print('Please enter a number from 1 to 3')
        while True:
            try:
                date = input('Enter Expenditure date in d/m/yyyy format:')
                date = datetime.strptime(date,'%d/%m/%Y')  #convert into datetime object
                break
            except ValueError:   #If invalid date format continue in the loo[]
                print('Invalid Date format!')
        while True:
            try:
                amount = float(input('Enter expenditure amount:$'))
                break
            except ValueError:  #If non-numerical value entered continue
                print('Invalid Amount!')
        print(f"{Expenditure__list.addExpenditure(date,amount,ex_type)} added")

    except ExpenditureException as e:  #If exception is catched then print the Error message with erro Type
        print(f"ERROR {e.errorType}: {e}")

def Option2(Expenditure__list):

    try:
        while True:
            try:
                choice = int(input('Enter Expenditure type (1-> Food, 2-> Transport, 3-> Education):'))
                #If choice is not 1/2/3 then continue else break out of while loop
                if choice!=1 and choice!=2 and choice!=3:
                    print('Type must be from 1 to 3')
                    continue
                if choice==1:
                    ex_type = 'Food'
                elif choice==2:
                    ex_type = 'Transport'
                else:
                    ex_type = 'Education'
                break
            except ValueError:  #if choice is a non-numerical value then continue
                print('Please enter a number from 1 to 3')
        while True:
            try:
                days = int(input('Expenditures for last how many days? 0 for all expenditures:'))
                break
            except ValueError:  #if days value entered is non-integer & non-numerical value continue
                print('Please enter a whole number for number of days')


        lis = Expenditure__list.getExpenditures(ex_type,days)  #Gets Expenditures of required Type and in required days
        if len(lis)==0:  #if blank list is returned then s variable be blank else add in the expenditures
            s=''
        else:
            s=''
            for x in lis:
                s+=f"\t{x}\n"
        print(s)
        print(f"Total: ${Expenditure__list.getExpendituresAmount(ex_type,days):0.2f}")
    except ExpenditureException as e:  #if an exception is catched then print the exception and error Type
        print(f"ERROR {e.errorType}: {e}")

def Option3(Expenditure__list):
    #This option lists all the expenditures, displaying both the subtotal and overall total.
    print(Expenditure__list)

def main():

    expenditure__list = ExpenditureList()

    while True:

        option = getMenu()

        if option == 0:       #if option is zero end the application
            print('\nApplication Ended')
            break
        elif option == 1:    #if an Expenditure object is created, it is added to expenditure__list
            Option1(expenditure__list)
        elif option == 2:     #list the expenditures incurred within the duration and of required Type
            Option2(expenditure__list)
        elif option == 3:     #option lists all the expenditures, displaying both the subtotal and overall total
            Option3(expenditure__list)

main()


Menu
1. Add Expenditure
2. List Expenditure by Type
3. List All Expenditures
0. Exit
Enter choice:3
Total for Food: $0.00
Total for Transport: $0.00
Total for Education: $0.00
Overall Total = $0.00

Menu
1. Add Expenditure
2. List Expenditure by Type
3. List All Expenditures
0. Exit
Enter choice:2
Enter Expenditure type (1-> Food, 2-> Transport, 3-> Education):1
Expenditures for last how many days? 0 for all expenditures:2

Total: $0.00
Menu
1. Add Expenditure
2. List Expenditure by Type
3. List All Expenditures
0. Exit
Enter choice:0

Application Ended


<br>
<h5>>>> 2(d)</h5>

Inheritance is used to allow sub-classes to inherit all its functions and methods extending the parent class into a more specialised sub-class.<br> It represents ‘is-a’ relationship. Instead of tirelessly rewriting same codes over and over again  inheritance provides code reusability.<br>
Whereas Object Composition represents ‘has-a’ relationship. A class would have one or more object of another class. It means that a class<br> referring to objects of another class, which are composites,  will also depend on those objects to execute certain function.<br>
In part (a), inheritance has been used by ExpenditureException class which inherits from Exception class. It is the specialised class which allow us<br>to raise specific exception demanded by this application. An exception from this class is raised whenever there is an error when recording an expenditure or when retrieving expenditure record(s) from the expenditure list.<br>
While using Expenditure class objects are created from it which undergo object composition in ExpenditureList class. Objects of Expenditure class are<br> acting as composites to be stored in a collection in ExpenditureList object. This enables ExpenditureList class to be encapsulated and focused on one task.


<br>
<h2 style="color:blue;text-align:center;">
    Question 3
</h2>
<br>

<h4>
    Develop the GUI using the tkinter framework. 
</h4>

<h5>>>> 3(a)</h5>

In [27]:
from tkinter import *
from tkinter.scrolledtext import ScrolledText

class ExpenditureGUI(Frame):

    def __init__(self):


        Frame.__init__(self)
        #-------------------------------Window Setting----------------------------------------

        self.master.title('Expenditure Tracker')
        self.master.geometry('600x198')

        #-----------------------------ExpenditureList-----------------------------------------

        self._expenditureList = [['Education',0],['Food',0],['Transport',0]]

        #--------------------------Creation Of Compnonets--------------------------------------

        self._labelType = Label(self,text='Type Of Expenditure:') #Type Of Expenditure Label

        #RadioButtons
        self._radioVar = StringVar()    #Variable for RadioButtons
        self._radioVar.set('E')
        self._educationRadio = Radiobutton(self,text='Education',value='E',variable=self._radioVar) #Education RadioButton
        self._foodRadio = Radiobutton(self,text='Food',value='F',variable=self._radioVar) #Food RadioButton
        self._transportRadio = Radiobutton(self,text='Transport',value='T',variable=self._radioVar) #Transport RadioButton

        self._labelAmount = Label(self,text="Amount:")   #Amount Label
        self._entryAmount =  Entry(self)                 #Amount Entry

        #Buttons
        self._addButton = Button(self,text='Add Expenditure',command=self.addExpenditure)  #Add Expenditure Button
        self._checkButton = Button(self,text='Check Expenditure',command=self.CheckExpenditure)  #Check Expenditure Button
        self._clearButton= Button(self,text='Clear Output',command=self.ClearOutput)    #Clear Output Button

        #Scrolled Text
        self._scrxltxt = ScrolledText(self,width=72,height=7)

        #------------------------------Placing Of Componenets-------------------------------------

        self._labelType.grid(row=0,column=0,padx=(90,0))
        self._educationRadio.grid(row=0,column=1,sticky=W)
        self._foodRadio.grid(row=0,column=1,sticky=E)
        self._transportRadio.grid(row=0,column=2,sticky=W)
        self._labelAmount.grid(row=1,column=0,padx=(30,0))
        self._entryAmount.grid(row=1,column=1,columnspan=2,sticky=E+W)
        self._addButton.grid(row=2,column=0,pady=4,sticky=E)
        self._checkButton.grid(row=2,column=1)
        self._clearButton.grid(row=2,column=2,sticky=W)
        self._scrxltxt.grid(row=3,column=0,columnspan=10,sticky=W)
        self._scrxltxt.configure(state=DISABLED)
        self.grid()

    def addExpenditure(self):

        self._scrxltxt.configure(state=NORMAL)
        try:
            _type = self._radioVar.get()   #Get the type of expenditure selected
            amount = float(self._entryAmount.get())   #Get the amount entered in Amount Entry
        except ValueError:  #If there is Value Error Catch here and execute the following
            value = self._entryAmount.get()
            if not value:  #if value is <Enter>
                self._scrxltxt.insert(END,f"You did not enter a value.Please enter a number for amount\n")
            else:
                #If value is non-numerical value then
                self._scrxltxt.insert(END,f"Invalid Amount:{value}.Please enter a number for amount\n")
                self._entryAmount.delete(0,END)
        else:
            #if its numerical value then we add it to the respective component
            for L in self._expenditureList:
                if _type.upper() == L[0][0]:
                    old=L[1]  #initial total value for that component
                    L[1]+=amount  #updates the value for that component
                    new=L[1]  #new total value for that component
                    status=L[0]
                    break

            self._scrxltxt.insert(END,f"${amount:0.2f} added to {status} ${old:0.2f} \n")
            self._scrxltxt.insert(END,f"New Total : ${new:0.2f}\n")
            self._entryAmount.delete(0,END) #Delete Amount Entry
        self._scrxltxt.configure(state=DISABLED)

    def CheckExpenditure(self):
        self._scrxltxt.configure(state=NORMAL)
        s="\n"
        all_list = self._expenditureList
        total=0  #Grand Total , sum of total of all component expenditures
        for expendit in all_list:
            s+=f"{expendit[0]:<12} Total: ${expendit[1]:0.2f}\n"
            total+=expendit[1]
        s+=f"Grand Total: ${total:0.2f}\n"
        self._scrxltxt.insert(END,f"{s}")
        self._scrxltxt.configure(state=DISABLED)

    def ClearOutput(self):
        #clears the content of the scrollable text pane
        self._scrxltxt.configure(state=NORMAL)
        self._scrxltxt.delete(1.0,END)
        self._scrxltxt.configure(state=DISABLED)

def main():

    gui = ExpenditureGUI()
    gui.mainloop()

main()


<br>
<h5>>>> 3(b)(i)</h5><br>
<i style="color:orange;text-decoration:underline;">Required Update</i>
<br>

In [28]:
from tkinter import *
from tkinter.scrolledtext import ScrolledText

class ExpenditureGUI(Frame):

    def __init__(self,listOfCategories):


        Frame.__init__(self)
        #-------------------------------Window Setting----------------------------------------

        self.master.title('Expenditure Tracker')
        self.master.geometry('650x198')

        #-----------------------------ExpenditureList-----------------------------------------

        self._listOfCategories = listOfCategories
        self._expenditureList = []
        for c in self._listOfCategories:
            self._expenditureList.append([c,0])


        #--------------------------Creation Of Compnonets--------------------------------------

        self._labelType = Label(self,text='Type Of Expenditure:') #Type of Expenditure Label


        #Radio Buttons
        self._radioVar = StringVar()    #Variable for RadioButtons
        self._radioVar.set(self._listOfCategories[0][0])
        self._radioList = []    #collection of all radio buttons in this GUI
        for radios in self._listOfCategories:
            self._radioList.append(Radiobutton(self,text=radios,value=radios[0],variable=self._radioVar))

        self._labelAmount = Label(self,text="Amount:")   #Amount Label
        self._entryAmount =  Entry(self)                 #Amount Entry

        #Buttons
        self._addButton = Button(self,text='Add Expenditure',command=self.addExpenditure)
        self._checkButton = Button(self,text='Check Expenditure',command=self.CheckExpenditure)
        self._clearButton= Button(self,text='Clear Output',command=self.ClearOutput)
        #Scrolled Text
        self._scrxltxt = ScrolledText(self,width=78,height=7)

        #------------------------------Placing Of Componenets-------------------------------------

        self._labelType.grid(row=0,column=0,padx=(60,0))
        #Placing Radio Buttons
        col = 1
        for radios in self._radioList:
            radios.grid(row=0,column=col)
            col+=1
        self._labelAmount.grid(row=1,column=0)
        self._entryAmount.grid(row=1,column=1,columnspan=len(self._listOfCategories),sticky=W+E)
        self._addButton.grid(row=2,column=1,pady=4)
        self._checkButton.grid(row=2,column=2)
        self._clearButton.grid(row=2,column=3)
        self._scrxltxt.grid(row=3,column=0,columnspan=10,sticky=W)
        self._scrxltxt.configure(state=DISABLED)
        self.grid()

    def addExpenditure(self):

        self._scrxltxt.configure(state=NORMAL)
        try:
            _type = self._radioVar.get()   #Get the type of expenditure selected
            amount = float(self._entryAmount.get())  #Get the amount entered in Amount Entry
        except ValueError:         #If there is Value Error Catch here and execute the following
            value = self._entryAmount.get()
            if not value:   #if value is <Enter>
                self._scrxltxt.insert(END,f"You did not enter a value.Please enter a number for amount\n")
            else:
                #If value is non-numerical value then
                self._scrxltxt.insert(END,f"Invalid Amount:{value}.Please enter a number for amount\n")
                self._entryAmount.delete(0,END)
        else:
            #if its numerical value then we add it to the respective component
            for L in self._expenditureList:
                if _type.upper() == L[0][0]:
                    old = L[1]    #initial total value for that component
                    L[1]+=amount  #updates the value for that component
                    new=L[1]       #new total value for that component
                    status=L[0]
                    break

            self._scrxltxt.insert(END,f"${amount:0.2f} added to {status} ${old:0.2f} \n")
            self._scrxltxt.insert(END,f"New Total : ${new:0.2f}\n")
            self._entryAmount.delete(0,END)
        self._scrxltxt.configure(state=DISABLED)

    def CheckExpenditure(self):
        self._scrxltxt.configure(state=NORMAL)
        s="\n"
        all_list = self._expenditureList
        total=0    #Grand Total , sum of total of all component expenditures
        for expendit in all_list:
            s+=f"{expendit[0]:<12} Total: ${expendit[1]:0.2f}\n"
            total+=expendit[1]
        s+=f"Grand Total: ${total:0.2f}\n"
        self._scrxltxt.insert(END,f"{s}")
        self._scrxltxt.configure(state=DISABLED)

    def ClearOutput(self):
        #clears the content of the scrollable text pane
        self._scrxltxt.configure(state=NORMAL)
        self._scrxltxt.delete(1.0,END)
        self._scrxltxt.configure(state=DISABLED)

def main():
    gui = ExpenditureGUI(['Food', 'Transport', 'Utility', 'Grocery', 'Medical'])
    gui.mainloop()

main()


<br>
<h5>>>> 3(b)(ii)</h5>

Exception handling is catching of an exception raised and dealing with it as per instructed by the developer whereas Event Handling is a mechanism that enables an event to be controlled and decides what should happen if an event occurs. This mechanism has the code which is known as event handler that is executed when an event occurs.<br><br>
<b><u>Two Examples of Event Handling in part(b):</u></b>
Firstly, in my program I handled Add Expenditure event, which adds the amount to specific expenditure type and showcase the result in the Scrolled Text, when “Add Expenditure” button is clicked. I defined the method addExpenditure to handle this event each time  “Add Expenditure” button is pressed.
Secondly, in my program I handled Check Expenditure event, which computes the grand total of all expenditure types amount and showcase the result in the Scrolled Text, when “Check Expenditure” button is clicked. I defined the method CheckExpenditure to handle this particular event each time “Check Expenditure”  button is pressed.<br>

<b><u>Two Examples of Exception Handling in part(b):</u></b><br>
Firstly, in my program under addExpenditure method I handled the exception for Value Error by catching it if value of amount entered is non-numerical value and then updating the user of an error committed by providing required messages in Scrolled Text.<br>
Secondly, I think I can implement an Exception Handling for the maximum number of items that can be allowed in the list of categories as eventually having large list of items will cause some of the radio buttons to be out of the screen as the size of GUI screen will not be adequate to cater for all components. This can be executed by catching the error under constructor and then inserting an error message in Scrolled Text to update the user of what went wrong, if an error is raised.
